# WGAN

can reference [here](https://github.com/KUASWoodyLIN/TF2-WGAN/blob/master/train.py)

and [here](https://github.com/timsainb/tensorflow2-generative-models/blob/master/3.0-WGAN-GP-fashion-mnist.ipynb)

[here](https://github.com/HCMY/TF2-GAN/blob/master/implementations/wgan/wgan.py)

Keywords :
1. 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.constraints import Constraint
from tensorflow.keras import layers
from tensorflow.keras import Model
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# from tqdm import tqdm
from pathlib import Path
import time
import PIL
from IPython import display

## A. Data Loading and Plotting

In [ ]:
# AnimeDataset: (96, 96, 3)
# extra_data: (64, 64, 3)
imgRootPath = Path('./dataset')
BSIZE = 32

In [ ]:
def parse_image(filename):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [64, 64])
    image = (image-0.5)*2.0
    return image

In [ ]:
train_dataset = tf.data.Dataset.list_files(str(imgRootPath/'**/*.jpg')).shuffle(buffer_size=70171)\
                    .map(parse_image,num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                    .batch(BSIZE).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for images in train_dataset:
    plt.imshow(images[0].numpy()/2.0 + 0.5)
    print(images.shape)
    break

In [ ]:
for images in train_dataset:
    print(np.max(images[0].numpy()))
    print(np.min(images[0].numpy()))
    break

## Define Generator and Discriminator

In [ ]:
# class Generator(Model):
#     def __init__(self):
#         super(Generator, self).__init__()
#         self.d1 = layers.Dense(16*16*128)
#         self.bn1 = layers.BatchNormalization()
#         self.relu1 = layers.ReLU()
        
#         self.reshape2 = layers.Reshape((16,16,128))
        
#         self.deconv3 = layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False)
#         self.bn3 = layers.BatchNormalization()
#         self.relu3 = layers.ReLU()
        
#         self.deconv4 = layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', use_bias=False)
#         self.bn4 = layers.BatchNormalization()
#         self.relu4 = layers.ReLU()
        
#         self.deconv5 = layers.Conv2DTranspose(3, (4, 4), strides=(1, 1), padding='same', use_bias=False)
        
#     def call(self,x):
#         x = self.d1(x)
#         x = self.bn1(x)
#         x = self.relu1(x)
        
#         x = self.reshape2(x)
        
#         x = self.deconv3(x)
#         x = self.bn3(x)
#         x = self.relu3(x)
        
#         x = self.deconv4(x)
#         x = self.bn4(x)
#         x = self.relu4(x)
        
#         x = self.deconv5(x)
        
#         return tf.keras.activations.tanh(x)

In [ ]:
def Generator(input_shape=(1, 1, 100), name='Generator'):
    inputs = keras.Input(shape=input_shape)

    # 1: 1x1 -> 4x4
    x = keras.layers.Conv2DTranspose(512, 4, strides=1, padding='valid', use_bias=False)(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)

    # 2: 4x4 -> 8x8
    x = keras.layers.Conv2DTranspose(256, 4, strides=2, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    # 3: 8x8 -> 16x16
    x = keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    # 4: 16x16 -> 32x32
    x = keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    # 5: 32x32 -> 64x64
    x = keras.layers.Conv2DTranspose(3, 4, strides=2, padding='same', use_bias=False)(x)
    outputs = keras.layers.Activation('tanh')(x)
    return keras.Model(inputs=inputs, outputs=outputs, name=name)

In [ ]:
generator = Generator()

In [ ]:
generator.build(input_shape=(None,100))
generator.summary()

In [ ]:
# class Discriminator(Model):
#     def __init__(self):
#         super(Discriminator, self).__init__()
#         self.conv1 = layers.Conv2D(32, 4, strides=(2, 2), padding='same')
#         self.bn1 = layers.BatchNormalization()
#         self.relu1 = layers.ReLU()
        
#         self.conv2 = layers.Conv2D(64, 4, strides=(2, 2), padding='same')
#         self.bn2 = layers.BatchNormalization()
#         self.relu2 = layers.ReLU()
        
#         self.conv3 = layers.Conv2D(128, 4, strides=(2, 2), padding='same')
#         self.bn3 = layers.BatchNormalization()
#         self.relu3 = layers.ReLU()
        
#         self.conv4 = layers.Conv2D(256, 4, strides=(2, 2), padding='same')
#         self.bn4 = layers.BatchNormalization()
#         self.relu4 = layers.ReLU()
        
#         self.flatten5 = layers.Flatten()
        
#         self.d6 = layers.Dense(1)
        
#     def call(self,x):
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu1(x)
        
#         x = self.conv2(x)
#         x = self.bn2(x)
#         x = self.relu2(x)
        
#         x = self.conv3(x)
#         x = self.bn3(x)
#         x = self.relu3(x)
        
#         x = self.conv4(x)
#         x = self.bn4(x)
#         x = self.relu4(x)
        
#         x = self.flatten5(x)
#         x = self.d6(x)
        
#         return x


In [ ]:
def Discriminator(input_shape=(64, 64, 3), name='Discriminator'):
    inputs = keras.Input(shape=input_shape)

    # 1: 64x64 -> 32x32
    x = keras.layers.Conv2D(64, 4, strides=2, padding='same')(inputs)
    x = keras.layers.LeakyReLU()(x)
    # 2: 32x32 -> 16x16
    x = keras.layers.Conv2D(128, 4, strides=2, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    # 3: 16x16 -> 8x8
    x = keras.layers.Conv2D(256, 4, strides=2, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    # 4: 8x8 -> 4x4
    x = keras.layers.Conv2D(512, 4, strides=2, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    # 5: 4x4 -> 1x1
    outputs = keras.layers.Conv2D(1, 4, strides=1, padding='valid')(x)
    return keras.Model(inputs=inputs, outputs=outputs, name=name)

In [ ]:
discriminator = Discriminator()

In [ ]:
discriminator.build(input_shape=(None,64,64,3))
discriminator.summary()

## Define the loss and optimizers

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = -tf.reduce_mean(real_output)
    fake_loss = tf.reduce_mean(fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
def generator_loss(fake_output):
    return -tf.reduce_mean(fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.RMSprop(1e-4)
discriminator_optimizer = tf.keras.optimizers.RMSprop(1e-4)
# generator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5, beta_2=0.9)
# discriminator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5, beta_2=0.9)
checkpoint_dir = Path('./training_checkpoints')
checkpoint_prefix = str(checkpoint_dir/"ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Define the training loop

In [ ]:
NOISE_DIM = 100
D_ITR = 5
EPOCHS = 700
clamp_lower = -0.01
clamp_upper = 0.01
num_examples_to_generate = 16
# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, 1, 1, NOISE_DIM])

In [ ]:
@tf.function
def train_dis_step(images):
    noise = tf.random.normal([BSIZE, 1, 1, NOISE_DIM])

    with tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    # apply weight clipping
    # discriminator.trainable_variables including bn?? Y
    for var in discriminator.trainable_variables:
#         tf.compat.v1.assign(var,tf.clip_by_value(var, clamp_lower, clamp_upper))
        var.assign(tf.clip_by_value(var, clamp_lower, clamp_upper))
#     ws = [tf.clip_by_value(w, clamp_lower, clamp_upper) for w in discriminator.get_weights()]
#     discriminator.set_weights(ws)


In [ ]:
# discriminator.trainable = True
# varlist = discriminator.trainable_variables
# names = [var.name for var in varlist]
# print(names)

# print(varlist[-1])
# varlist = discriminator.trainable_variables
# varlist[-1] = tf.clip_by_value(varlist[-1], -0.01, 0.01)
# print(varlist[-1])
# print(discriminator.trainable_variables[-1])

# ws = discriminator.get_weights()
# print(ws[-1])
# ws[-1] = tf.clip_by_value(ws[-1], -0.01, 0.01)
# print(ws[-1])
# discriminator.set_weights(ws)
# print(discriminator.get_weights()[-1])

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_gen_step():
    noise = tf.random.normal([BSIZE, 1, 1, NOISE_DIM])
    with tf.GradientTape() as gen_tape:
        generated_images = generator(noise, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        
        counter = 1
        for image_batch in dataset:
            train_dis_step(image_batch)
            ws = discriminator.get_weights()
#             print(ws[-1])
            if counter%D_ITR==0:
                train_gen_step()
            counter += 1

        # Produce images for the GIF as we go
        display.clear_output(wait=True)
        generate_and_save_images(generator,
                                 epoch + 1,
                                 seed)

        # Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                           epochs,
                           seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(10,10))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i].numpy()/2.0 + 0.5)
        plt.axis('off')

    plt.savefig('out_images/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [ ]:
train(train_dataset, EPOCHS)

## Create a GIF

In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
    return PIL.Image.open('out_images/image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(446)

Use `imageio` to create an animated gif using the images saved during training.

In [ ]:
import imageio, glob

In [ ]:
anim_file = 'all_rms_wgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = glob.glob('out_images/image*.png')
    filenames = sorted(filenames)
    last = -1
    for i,filename in enumerate(filenames):
        frame = 10*(i**0.5)
        if round(frame) > round(last):
            last = frame
        else:
            continue
        image = imageio.imread(filename)
        writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

import IPython
if IPython.version_info > (6,2,0,''):
    display.Image(filename=anim_file)

In [ ]:
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download(anim_file)

Try generating other images

In [ ]:
codes = np.random.randn(num_examples_to_generate, 1, 1, NOISE_DIM)
predictions = generator(codes, training=False)

fig = plt.figure(figsize=(10,10))

for i in range(predictions.shape[0]):
    plt.subplot(4, 4, i+1)
    plt.imshow(predictions[i].numpy()/2.0 + 0.5)
    plt.axis('off')